## Import libs

In [79]:
from typing import Tuple, List, Iterable
import time

import pandas as pd
import numpy as np
import sqlite3

PATH_TO_DB = ''

## Import data

In [8]:
bg = pd.read_feather('../Board-Games/data/full/bgg_boardgames.feather')

bg.boardgame_id = bg.boardgame_id.astype('int')

## Logic

To begin with, we will implement the simplest version of the recommendations for interacting with the chatbot, both at a cold-start and for users with an existing history.

In either case, it's worth getting input for making recommendations:
- Number of players:
    - solo
    - for two
    - 3-4
    - 5+

- Game duration:
    - up to 30 minutes
    - 30 - 90 minutes
    - 1.5 - 3 hours
    - more than 3 hours
    
- How long ago the game was released:
    - up to 3 years ago (newest)
    - 3-7 years ago (recent)
    - older than 7 years ago (older)
    - before 2007 (oldschool games)
    
- Preferred genres (?):
    - ...
    - ...
    
And later with these filters we will recommend games with the highest rank on BGG

## Realization

In [11]:
# Relevant values
n_players_v = ['solo', 'two', '3-4', '5+']
duration_v = ['up to 30m', '30-90m', '1.5 - 3h', '3h+']
game_age_v = ['newest', 'recent', 'older', 'oldschool']

In [84]:
def query_to_db(query: str, PATH_TO_DB: str): # -> Iterable[Tuple[...]]
    '''
    Retrieve data from the database on request
    
    ----------
    Parameters:
    query: str
        Query for getting data from database
        
    PATH_TO_DB: str
        Path to database
    ----------
    Return:
        List of tuples with data from database
    '''
    # connect to the database
    conn = sqlite3.connect(PATH_TO_DB)

    # create a cursor object
    cursor = conn.cursor()
    
    # get data
    cursor.execute(query)
    
    # collect data
    data = cursor.fetchall()
    
    # close the connection
    conn.close()

    return data



def huristic_recommendation(n_players: str, duration: str, game_age: str, n_recommend=10): # -> Iterable[Tuple[int, str]]
    '''
    Get recommendations based on heuristic: games with the highest rank on BGG with used terms.
    
    ----------
    Parameters:
    n_players: str
        Number of players. Must be one of this values: 'solo', 'two', '3-4', '5+'.
    
    duration: str
        Duration of game. Must be one of this values: 'up to 30m', '30-90m', '1.5 - 3h', '3h+'.
    
    game_age: str
        How old game was published. Must be one of this values: 'newest', 'recent', 'older', 'oldschool'
    
    n_recommend: int
        Number of games for recomendation. Must be more positive.
    
    PATH_TO_DB: str
        Path to database
    ----------
    Return:
        List of tuples with data from database
    '''
    assert n_players in n_players_v, 'Invalid number of players'
    assert duration in duration_v, 'Invalid game duration'
    assert game_age in game_age_v, 'Invalid age of the game'
    assert type(n_recommend) == int, 'Number of recommends is not integer'
    assert n_recommend > 0, 'Number of recommends is not positive integer'
    
    if n_players == 'solo':
        q_1 = '(minplayers = 1)'
    elif n_players == 'two':
        q_1 = '((minplayers <= 2) & (maxplayers >= 2))'
    elif n_players == '3-4':
        q_1 = '((minplayers <= 3) & (maxplayers >= 4))'
    elif n_players == '5+':
        q_1 = '(maxplayers >+ 5)'
    assert type(q_1) == str
    
    if duration == 'up to 30m':
        q_2 = '(maxplaytime <= 30)'
    elif duration == '30-90m':
        q_2 = '((maxplaytime >= 30) & (maxplaytime <= 90))'
    elif duration == '1.5 - 3h':
        q_2 = '((maxplaytime >= 90) & (maxplaytime <= 180))'
    elif duration == '3h+':
        q_2 = '(maxplaytime >= 180)'
    assert type(q_2) == str
    
    if game_age == 'newest': # up to 3 years ago
        q_3 = f'(year_published > {time.localtime().tm_year - 3})'
    elif game_age == 'recent': # 3-7 years ago
        q_3 = f'((year_published <= {time.localtime().tm_year - 3}) & (year_published > {time.localtime().tm_year - 7}))'
    elif game_age == 'older': # older than 7 years ago
        q_3 = f'((year_published <= {time.localtime().tm_year - 7}) & (year_published > 2007))'
    elif game_age == 'oldschool': # before 2007
        q_3 = f'(year_published <= 2007)'
    assert type(q_3) == str
    
    where_query = f'WHERE {q_1} & {q_2} & {q_3}'
    limit_query = f'LIMIT {n_recommend}'
    
    main_query = f'''
        SELECT b.boardgame_id, b.title
        FROM   (SELECT boardgame_id, title
                FROM boardgames
                {where_query}) b
        JOIN ranks r ON b.boardgame_id = r.boardgame_id
        ORDER BY r.bgg_rank ASC
        {limit_query}
        '''
    
#     return query_to_db(main_query, PATH_TO_DB=PATH_TO_DB)
    return main_query 

print(huristic_recommendation(n_players='3-4', duration='1.5 - 3h', game_age='recent', n_recommend=10))



        SELECT b.boardgame_id, b.title
        FROM   (SELECT boardgame_id, title
                FROM boardgames
                WHERE ((minplayers <= 3) & (maxplayers >= 4)) & ((maxplaytime >= 90) & (maxplaytime <= 180)) & ((year_published <= 2020) & (year_published > 2016))) b
        JOIN ranks r ON b.boardgame_id = r.boardgame_id
        ORDER BY r.bgg_rank ASC
        LIMIT 10
        


In [74]:
bg[bg.boardgame_id == 268620].ranks

61703    Board Game Rank|1379||Family Game Rank|387
Name: ranks, dtype: object